# Các bước thực hiện
* Compile các lỗ hổng từ source code thành bytecode
    * Nén dữ liệu từng lỗ hổng thành csv file tương ứng
* Xử lý dữ liệu csv: 
    * Đọc file dữ liệu cũ
    * Đọc từng file lỗ hổng riêng lẻ, xóa các dòng bị trùng lặp 
    * Xóa các dòng không compile được, dữ liệu bắt đầu với'\n'
    * Kiểm tra dữ liệu Outdated với bộ Outdated trong dữ liệu cũ
    * Gán label mới tương ứng với địa chỉ bytecode
    * Kết hợp dữ liệu Bytecode và dữ liệu Nhãn mới thành file dataset mới

# Import necessary lib

In [1]:
import pandas as pd 
import os 
import numpy as np
from tqdm import tqdm

In [2]:
# address_check = df1.loc[df1['Outdated Solidity version'] == 1, ['ADDRESS']]['ADDRESS'].tolist()
# address = df1['ADDRESS'].tolist()


# Read data raw

In [3]:
df_data = pd.read_csv('Data_Cleansing.csv')
address = df_data['ADDRESS'].tolist()

In [33]:
df_reentrancy = pd.read_csv("reentrancy.csv").drop_duplicates()
df_outdated = pd.read_csv("outdated.csv").drop_duplicates()
df_delegate = pd.read_csv("delegate.csv").drop_duplicates()
df_frozen = pd.read_csv("frozenether.csv").drop_duplicates()
df_timestamp = pd.read_csv("timestamp.csv").drop_duplicates()

### Clear noise in data 

In [35]:
df_reentrancy = df_reentrancy[~df_reentrancy['BYTECODE'].str.startswith('\n')]
df_outdated = df_outdated[~df_outdated['BYTECODE'].str.startswith('\n')]
df_delegate = df_delegate[~df_delegate['BYTECODE'].str.startswith('\n')]
df_frozen = df_frozen[~df_frozen['BYTECODE'].str.startswith('\n')]
df_timestamp = df_timestamp[~df_timestamp['BYTECODE'].str.startswith('\n')]

In [38]:
df_reentrancy['ADDRESS'] = df_reentrancy['ADDRESS'].str.replace(".txt","")
df_outdated['ADDRESS'] = df_outdated['ADDRESS'].str.replace(".txt","")
df_delegate['ADDRESS'] = df_delegate['ADDRESS'].str.replace(".txt","")
df_frozen['ADDRESS'] = df_frozen['ADDRESS'].str.replace(".txt","")
df_timestamp['ADDRESS'] = df_timestamp['ADDRESS'].str.replace(".txt","")

/tmp/ipykernel_13898/322731642.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_reentrancy['ADDRESS'] = df_reentrancy['ADDRESS'].str.replace(".txt","")
/tmp/ipykernel_13898/322731642.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_outdated['ADDRESS'] = df_outdated['ADDRESS'].str.replace(".txt","")
/tmp/ipykernel_13898/322731642.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df_delegate['ADDRESS'] = df_delegate['ADDRESS'].str.replace(".txt","")
/tmp/ipykernel_13898/322731642.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df_frozen['ADDRESS'] = df_frozen['ADDRESS'].str.replace(".txt","")
/tmp/ipykernel_13898/322731642.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_timestamp['ADDRESS'] = df_timestamp['ADDRESS'].s

### Check data

In [54]:
addr_reentrancy = df_reentrancy['ADDRESS'].tolist()
print(f'Numbers of Reentrancy address: {len(addr_reentrancy)}')

addr_outdated = df_outdated['ADDRESS'].tolist()
print(f'Numbers of Outdated address: {len(addr_outdated)}')

addr_delegate = df_delegate['ADDRESS'].tolist()
print(f'Numbers of Delegate address: {len(addr_delegate)}')

addr_frozen = df_frozen['ADDRESS'].tolist()
print(f'Numbers of FrozenEther: {len(np.unique(addr_frozen))}')

addr_timestamp = df_timestamp['ADDRESS'].tolist()
print(f'Numbers of Timestamp: {len(np.unique(addr_timestamp))}')

Numbers of Reentrancy address: 88554
Numbers of Outdated address: 105529
Numbers of Delegate address: 35132
Numbers of FrozenEther: 96981
Numbers of Timestamp: 54666


In [56]:
data_reen = df_reentrancy['BYTECODE'].tolist()
print(f'Numbers of Reentrancy data: {len(data_reen)}')

data_outdated = df_outdated['BYTECODE'].tolist()
print(f'Numbers of Outdated data: {len(data_outdated)}')

data_delegate = df_delegate['BYTECODE'].tolist()
print(f'Numbers of Delegate data: {len(data_delegate)}')

data_frozen = df_frozen['BYTECODE'].tolist()
print(f'Numbers of Frozen data: {len(data_frozen)}')

data_timestamp = df_timestamp['BYTECODE'].tolist()
print(f'Numbers of Timestamp data: {len(data_timestamp)}')

Numbers of Reentrancy data: 88554
Numbers of Outdated data: 105529
Numbers of Delegate data: 35132
Numbers of Frozen data: 96981
Numbers of Timestamp data: 54666


In [6]:
def compare(addr1,addr2):   
    set1 = set(addr1)
    set2 = set(addr2)

    commons = set1.intersection(set2)

    com_lists = list(commons)
    print("Comparing matching")
    print(len(com_lists))

### Check Outdated data

In [43]:
data_check = df_data.loc[df_data['Outdated Solidity version'] == 1, ['BYTECODE']]['BYTECODE'].tolist()
data_true = df_outdated['BYTECODE'].tolist()

compare(data_check, data_true)
print('Outdated in old dataset')
print(len(data_check))
print('Actual outdated data after cleansing')
print(len(data_true))

Comparing matching
105518
Outdated in old dataset
118863
Actual outdated data after cleansing
105529


# Create dataframe include address and vulnerabilities label


address Reen -> address Delegate -> address Frozen -> address Outdated -> address Times

In [47]:
LABEL = ['ADDRESS', 'BYTECODE']
VUL_LABEL = ['Reentrancy', 'Delegatecall injection', 'Frozen Ether', 'Outdated Solidity version', 'Timestamp Dependency']
columns = LABEL+VUL_LABEL
columns

['ADDRESS',
 'BYTECODE',
 'Reentrancy',
 'Delegatecall injection',
 'Frozen Ether',
 'Outdated Solidity version',
 'Timestamp Dependency']

combine address into unique list

In [ ]:
combined_address = list(set(addr_reentrancy+addr_delegate+addr_frozen+addr_outdated+addr_timestamp))

print(len(combined_address))

In [49]:
def get_vul_label(address, address_reentrancy, address_delegate, address_frozen, address_outdated, address_timestamp):
    vul_label = []
    for item in tqdm(address):
        # print(f'{index+1}/{total}')
        if item in address_reentrancy:
            v1 = 1
        else:
            v1 = 0
        
        if item in address_delegate:
            v2 = 1
        else:
            v2 = 0

        if item in address_frozen:
            v3 = 1
        else:
            v3 = 0
        
        if item in address_outdated:
            v4 = 1
        else:
            v4 = 0
        
        if item in address_timestamp:
            v5 = 1
        else:
            v5 = 0
        # print(f'{(v1,v2,v3,v4,v5)}')
        vul_label.append((v1,v2,v3,v4,v5))
    return vul_label

In [68]:
vul_label = get_vul_label(combined_address, addr_reentrancy, addr_delegate,addr_frozen, addr_outdated, addr_timestamp)

100%|██████████| 156731/156731 [14:27<00:00, 180.74it/s]


In [72]:
vul_label[0]

(0, 1, 1, 1, 0)

In [82]:
label_df = pd.DataFrame(vul_label, columns=VUL_LABEL)

In [83]:
label_df['ADDRESS'] = combined_address

In [84]:
label_df

,Reentrancy,Delegatecall injection,Frozen Ether,Outdated Solidity version,Timestamp Dependency,ADDRESS
0,0,1,1,1,0,0xbdf5a9920f8fa95f46bc53d568b0156af33e37c3
1,1,0,0,1,1,0x7b87901d64364f56268fb8004e003295d79bd9b7
2,0,1,1,0,0,0x599d7ec508a001aced11fcca7bbd4b522bed6d6e
3,1,0,0,1,1,0xe906a32973a5780cd477fb550b9a78c93b2506df
4,1,0,0,1,1,0xde642b8e2571f7bcf028a0ab4d9fbfe59a2c9525
...,...,...,...,...,...,...
156726,0,0,1,0,0,0x6305fb0ce7a6e99d18709d8b54c98081e772bfe6
156727,0,1,1,0,0,0x1870966e060856136beb293438bae5e30d43601c
156728,1,0,0,1,1,0x698713b58f4385ba62bd721a72618421f086d5d4
156729,0,0,1,0,0,0x2549678bce4240de0b7e41a821d5157ce80e45e9


# Create Dataframes with Bytecode and Address

In [67]:
dataframes = [df_reentrancy, df_delegate, df_frozen, df_outdated, df_timestamp]
combined_df = pd.concat(dataframes)
combined_df = combined_df.drop_duplicates(subset='ADDRESS')

combined_df

,BYTECODE,ADDRESS
0,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x2f9fe0851eb80bd8ae3450b896f1cd6a0c1e06ab
2,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x263b7132a315cf1fb266fff3d416f31e4d2d3899
4,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x4478535df69f9d9b151b079c21a56a1c6266ef82
5,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0xc391f26e4de429d3177980e456e68f266bb66c3a
6,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0xc5af6ff1e81f86b13583f6a607c21e5f3b55d794
...,...,...
58451,60 56 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x1dc3c2c16195bd6e49f6ed05f1f482c3c7fd35af
58534,60 80 60 40 52 34 80 15 61 00 10 57 60 00 80 f...,0x6e40a9d1ee2c8ef95322b879cbae35be6dd2d143
58715,60 80 60 40 52 34 80 15 61 00 10 57 60 00 80 f...,0xb7d018f57b82d1523f15a270d6b942836204310b
58810,60 56 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x31932e6e45012476ba3a3a4953cba62aee77fbbe


# Merge dataframe by address

In [85]:
df_new_dataset = pd.merge(combined_df,label_df,on='ADDRESS', how='left')

In [88]:
df_new_dataset.to_csv('BytecodeMultivulDataset.csv', index=False)

# Split


In [2]:
new_df = pd.read_csv('BytecodeMultivulDataset.csv')

In [11]:
new_df.head()

,BYTECODE,ADDRESS,Reentrancy,Delegatecall injection,Frozen Ether,Outdated Solidity version,Timestamp Dependency
0,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x2f9fe0851eb80bd8ae3450b896f1cd6a0c1e06ab,1,0,0,1,1
1,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x263b7132a315cf1fb266fff3d416f31e4d2d3899,1,0,0,1,1
2,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0x4478535df69f9d9b151b079c21a56a1c6266ef82,1,0,0,1,1
3,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0xc391f26e4de429d3177980e456e68f266bb66c3a,1,0,0,1,1
4,60 55 60 23 60 0b 82 82 82 39 80 51 60 00 1a 6...,0xc5af6ff1e81f86b13583f6a607c21e5f3b55d794,1,0,0,1,1


In [15]:
LABEL = ['BYTECODE', 'ADDRESS']
VUL_LABEL = ['Reentrancy', 'Delegatecall injection', 'Frozen Ether', 'Outdated Solidity version', 'Timestamp Dependency']
columns = LABEL+VUL_LABEL

In [8]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(data, train_size=0.8)

In [2]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')